In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import time

In [2]:
HEADER = {"User-Agent":"Chrome/88.0.4324.182"}

def get_cover_urls(sitemap_url="https://www.magazine-data.com/sitemap/sitemap.xml"):
    """
    表紙情報のurl一覧を取得する関数です。
    Args:
        sitemap_url (str) : サイトマップのurl
    """
    
    res = requests.get(sitemap_url, headers=HEADER)
    soup = BeautifulSoup(res.text, "html.parser")
    
    cover_urls = []

    flag = False
    for url in soup.select("url > loc"):
        if re.search(r"/cover/", url.text): # coverを含むもの処理
            if url.text == "https://www.magazine-data.com/cover/anan.html": # anan以降を取得
                flag = True
            if not flag:
                continue
            else:
                cover_urls.append(url.text)
                
    return cover_urls


def get_magazine_summary(url):
    res = requests.get(url, headers=HEADER)
    soup = BeautifulSoup(res.text, "html.parser")
    
    boxes = soup.select_one("#furoku_box dl")
    datas = {"巻号":[], "モデル":[], "URL":[], "発売日":[]}
    
    # 巻号と雑誌のurlを取得
    for cover in boxes.find_all("dt"):
        if cover.find("a"):
            magazine_url = cover.a.attrs["href"]
        else:
            magazine_url = url
        magazine_id = " ".join([w for w in cover.text.split() if w != "→"][1:])

        datas["巻号"].append(magazine_id)
        datas["URL"].append(magazine_url)

    # モデルと発売日を取得
    for model_data in boxes.select("dd p.model"):
        model, release_date = model_data.text.split("発売日：")
        model = model.split("：")[-1]

        datas["モデル"].append(model)
        datas["発売日"].append(release_date)
        
    df = pd.DataFrame(datas)
    
    # titleから雑誌名を取得
    magazine_name = soup.title.text.split("【表紙】")[0].split()
    if len(magazine_name) == 2:
        name_en, name_ja = magazine_name # 英字の雑誌なら日本語表記を取得
    else:
        name_en = magazine_name[0]
        name_ja = name_en # 日本語の雑誌はそのまま
        
    df["雑誌名"] = name_en
    df["ザッシ名"] = name_ja
    df["年"] = df["発売日"].map(lambda x: int(re.search(r"[0-9]+年", x).group()[:-1]))
    df["月"] = df["発売日"].map(lambda x: int(re.search(r"[0-9]+月", x).group()[:-1]))
    
    return df

In [3]:
cover_urls = get_cover_urls()

In [4]:
cover_urls

['https://www.magazine-data.com/cover/anan.html',
 'https://www.magazine-data.com/cover/nonno.html',
 'https://www.magazine-data.com/cover/popteen.html',
 'https://www.magazine-data.com/cover/seventeen.html',
 'https://www.magazine-data.com/cover/cancam.html',
 'https://www.magazine-data.com/cover/vivi.html',
 'https://www.magazine-data.com/cover/ray.html',
 'https://www.magazine-data.com/cover/jj.html',
 'https://www.magazine-data.com/cover/ar.html',
 'https://www.magazine-data.com/cover/mini.html',
 'https://www.magazine-data.com/cover/mina.html',
 'https://www.magazine-data.com/cover/with.html',
 'https://www.magazine-data.com/cover/more.html',
 'https://www.magazine-data.com/cover/sweet.html',
 'https://www.magazine-data.com/cover/steady.html',
 'https://www.magazine-data.com/cover/spring.html',
 'https://www.magazine-data.com/cover/ginger.html',
 'https://www.magazine-data.com/cover/andgirl.html',
 'https://www.magazine-data.com/cover/bijinhyakka.html',
 'https://www.magazine-data

In [5]:
len(cover_urls)

105

雑誌は全部で105件

In [6]:
# 特定の雑誌情報の取得
df = get_magazine_summary(cover_urls[0])
df

,巻号,モデル,URL,発売日,雑誌名,ザッシ名,年,月
0,2021年 3/10号,菊池 風磨,https://www.magazine-data.com/page/20210303ana...,2021年3月3日,an・an,アンアン,2021,3
1,2021年 3/3号,日向坂46,https://www.magazine-data.com/page/20210224ana...,2021年2月24日,an・an,アンアン,2021,2
2,2021年 2/24号,岩田 剛典・新田 真剣佑,https://www.magazine-data.com/page/20210217ana...,2021年2月17日,an・an,アンアン,2021,2
3,2021年 2/17号,松村 北斗,https://www.magazine-data.com/page/20210210ana...,2021年2月10日,an・an,アンアン,2021,2
4,2021年 2/10号,玉森 裕太,https://www.magazine-data.com/page/20210203ana...,2021年2月3日,an・an,アンアン,2021,2
...,...,...,...,...,...,...,...,...
95,2019年 3/6号,増田 貴久,https://www.magazine-data.com/cover/anan.html,2019年2月27日,an・an,アンアン,2019,2
96,2019年 2/27号,KAT-TUN,https://www.magazine-data.com/cover/anan.html,2019年2月20日,an・an,アンアン,2019,2
97,2019年 2/20号,高木 雄也,https://www.magazine-data.com/cover/anan.html,2019年2月13日,an・an,アンアン,2019,2
98,2019年 2/13号,登坂 広臣,https://www.magazine-data.com/cover/anan.html,2019年2月6日,an・an,アンアン,2019,2


In [7]:
# 全雑誌情報の取得
# sleep内に指定した時間×雑誌数くらいの時間がかかります。
for i, url in enumerate(cover_urls):
    if i == 0:
        df = get_magazine_summary(url)
    else:
        df = pd.concat([df, get_magazine_summary(url)])
    time.sleep(2) # サイトに負荷をかけないように2秒待つ。少なくとも1秒以上に設定してください。

In [8]:
df

,巻号,モデル,URL,発売日,雑誌名,ザッシ名,年,月
0,2021年 3/10号,菊池 風磨,https://www.magazine-data.com/page/20210303ana...,2021年3月3日,an・an,アンアン,2021,3
1,2021年 3/3号,日向坂46,https://www.magazine-data.com/page/20210224ana...,2021年2月24日,an・an,アンアン,2021,2
2,2021年 2/24号,岩田 剛典・新田 真剣佑,https://www.magazine-data.com/page/20210217ana...,2021年2月17日,an・an,アンアン,2021,2
3,2021年 2/17号,松村 北斗,https://www.magazine-data.com/page/20210210ana...,2021年2月10日,an・an,アンアン,2021,2
4,2021年 2/10号,玉森 裕太,https://www.magazine-data.com/page/20210203ana...,2021年2月3日,an・an,アンアン,2021,2
...,...,...,...,...,...,...,...,...
17,blue 2019年5月号,吉沢 亮,https://www.magazine-data.com/cover/auditionbl...,2019年4月1日,Audition,Audition,2019,4
18,blue 2019年4月号,片寄 涼太,https://www.magazine-data.com/cover/auditionbl...,2019年3月1日,Audition,Audition,2019,3
19,blue 2019年3月号,新田 真剣佑,https://www.magazine-data.com/cover/auditionbl...,2019年2月1日,Audition,Audition,2019,2
20,blue 2019年2月号,BOYS AND MEN,https://www.magazine-data.com/cover/auditionbl...,2018年12月29日,Audition,Audition,2018,12


In [9]:
df["モデル"].value_counts()

King & Prince     64
SixTONES          61
Snow Man          57
嵐                 38
ジャニーズ WEST        30
                  ..
ラブライブ!サンシャイン!!     1
諏訪 ななか             1
山﨑 天               1
与田 祐希・ 山下 美月       1
長澤 まさみ・東出 昌大       1
Name: モデル, Length: 916, dtype: int64

In [10]:
df[df["年"]==2021]["モデル"].value_counts()

松村 北斗                9
Snow Man             9
SixTONES             6
亀梨 和也                5
玉森 裕太                4
                    ..
滝沢 カレン               1
石原 さとみ               1
トリンドル 玲奈・トリンドル 瑠奈    1
桐谷 美玲                1
茅野 愛衣                1
Name: モデル, Length: 166, dtype: int64

In [12]:
df.to_csv("210225_analyze_magazine_cover.csv", index=False)